In [1]:
!pip install "git+git://github.com/bblais/Game" --upgrade

  Cloning git://github.com/bblais/Game to c:\users\student\appdata\local\temp\pip-req-build-fqihoez1


  Running command git clone -q git://github.com/bblais/Game 'C:\Users\student\AppData\Local\Temp\pip-req-build-fqihoez1'


In [2]:
from Game import*

Version:  0.2.35


In [3]:
Card?

Init signature: Card(rank, suit)
Docstring:      <no docstring>
File:           c:\users\student\anaconda3\lib\site-packages\game\cards.py
Type:           type
Subclasses:     


In [4]:
deck=makedeck()
deck

['JC',
 'AD',
 '5H',
 '2D',
 '6S',
 '6D',
 'QS',
 '3H',
 '6C',
 '8S',
 '10H',
 'KS',
 '2H',
 'AH',
 '7C',
 '3C',
 '7S',
 '9C',
 '8C',
 '3D',
 '9H',
 'AS',
 '2S',
 '4H',
 '3S',
 'JS',
 'KD',
 '5D',
 '10D',
 'QC',
 '7H',
 '4S',
 'KC',
 '2C',
 '5S',
 'KH',
 '10C',
 '4C',
 '9D',
 '8D',
 '7D',
 'JD',
 '6H',
 '5C',
 '10S',
 '4D',
 '8H',
 'QD',
 'JH',
 'AC',
 'QH',
 '9S']

In [5]:
print(cards)
print(len(deck),deck)

<module 'Game.cards' from 'C:\\Users\\student\\anaconda3\\lib\\site-packages\\Game\\cards.py'>
52 ['JC', 'AD', '5H', '2D', '6S', '6D', 'QS', '3H', '6C', '8S', '10H', 'KS', '2H', 'AH', '7C', '3C', '7S', '9C', '8C', '3D', '9H', 'AS', '2S', '4H', '3S', 'JS', 'KD', '5D', '10D', 'QC', '7H', '4S', 'KC', '2C', '5S', 'KH', '10C', '4C', '9D', '8D', '7D', 'JD', '6H', '5C', '10S', '4D', '8H', 'QD', 'JH', 'AC', 'QH', '9S']


In [6]:
def valid_moves(state,player):
    return["hit","stay"]

def initial_state():
    state = []
    player1=deal(deck,2)
    state.append(player1)
    dealer=deal(deck,1)
    state.append(dealer)
    return state

In [7]:
state = initial_state()
state

[['JC', 'AD'], ['5H']]

In [51]:
def show_state(state):
    print("The players hands are",state)

In [52]:
show_state(state)

The players hands are [['JC', 'AD'], ['5H']]


In [67]:
def update_state(state,move,player):
    new_state=state
    player_total, dealer_total, turn_total, =state
    
    if move=="hit":
        card=deal(deck,1)
        
    else:
        if player==1:
            new_state[0]+=turn_total
        else:
            new_state[1]+=turn_total
    if move=="stay":
        return True
            

In [68]:
def win_status(state,player):
    max_score=21
    
    if player==1:
        if player_total>max_score:
            return "lose"
    elif player==2:
        if dealer_total>max_score:
            return "lose"
    elif player==1:
        if player_total<dealer_total:
            return "lose"
    elif player==2:
        if dealer_total<player_total:
            return "lose"
    elif player==1:
        if player_total>dealer_total and player_total>max_score:
            return "lose"
    elif player==2:
        if dealer_total>player_total and dealer_total>max_score:
            return "lose"
    elif player==1:
        if player_total>dealer_total and player_total<=max_score:
            return "win"
    elif player==2:
        if dealer_total>player_total and dealer_total<=max_score:
            return "win"
    else:
        if dealer_total==player_total and dealer_total<=max_score and player_total<=max_score:
            return "tie"
    

In [69]:
def score(cards):
    
    total=0
    for card in cards:
        
        if card.rank==13 or card.rank==12 or card.rank==11:
            total+=10
        else:
            total+=card.rank
    
    return total

In [70]:
def human_move(state,player):
    print("Player", player)
    s=input("hit or stay?")
    
    if s[0]=='h':
        return "hit"
    elif s[0]=='s':
        return "stay"

human_agent=Agent(human_move)

In [71]:
def random_move(state,player):
    move=random.choice(valid_moves(state,player))
    return move

random_agent=Agent(random_move)

In [72]:
from Game.minimax import *
def minimax_move(state,player):
    values,moves=minimax_values(state,player,display=True)
    return top_choice(moves,values)

minimax_agent=Agent(minimax_move)

In [73]:
def skittles_move(state,player,info):
    T=info.T
    last_state=info.last_state
    last_action=info.last_action
    learning=info.learning
    
    if state not in T:
        actions=valid_moves(state,player)
        T[state]=Table()
        for action in actions:
            T[state][action]=2 
    
    move=weighted_choice(T[state])
    
    if move is None:
        
        # learn
        if learning:
            if last_state:
                T[last_state][last_action]-=1.
                if T[last_state][last_action]<0:
                    T[last_state][last_action]=0
        
        return random_move(state,player)
    else:
        return move

In [74]:
def skittles_after(status,player,info):
    T=info.T
    last_state=info.last_state
    last_action=info.last_action
    learning=info.learning
    
    if status=='lose': 
        T[last_state][last_action]-=1. 
        if T[last_state][last_action]<0:
            T[last_state][last_action]=0
    pass

In [75]:
skittles_agent=Agent(skittles_move)
skittles_agent.post=skittles_after
skittles_agent.T=Table()   
skittles_agent.learning=True

In [76]:
def Q_move(state,player,info):
    Q=info.Q
    last_state=info.last_state
    last_action=info.last_action
    learning=info.learning
    
    α=info.α 
    ϵ=info.ϵ 
    γ=info.γ 
    
    # \alpha <hit tab>    α
    # \epsilon <hit tab>  ϵ
    # \gamma <hit tab>    γ
    
    if state not in Q:
        actions=valid_moves(state,player)
        Q[state]=Table()
        for action in actions:
            Q[state][action]=0  
    
    
    if random.random()<ϵ:  
        move=random_move(state,player)
    else:
        move=top_choice(Q[state])
    
    if not last_action is None:  
        reward=0
        
        # learn
        if learning:
            Q[last_state][last_action]+=α*(reward +
                        γ*max([Q[state][a] for a in Q[state]]) - Q[last_state][last_action])
    return move

In [77]:
def Q_after(status,player,info):
    Q=info.Q
    last_state=info.last_state
    last_action=info.last_action
    learning=info.learning
    
    α=info.α  
    ϵ=info.ϵ  
    γ=info.γ  
    
    # \alpha <hit tab>    α
    # \epsilon <hit tab>  ϵ
    # \gamma <hit tab>    γ

    if status=='lose':
        reward=-1
    elif status=='win':
        reward=1
    elif status=='stalemate':
        reward=.5 
    else:
        reward=0
    
    
    if learning:
        Q[last_state][last_action]+=α*(reward - Q[last_state][last_action])

In [78]:
Q1_agent=Agent(Q_move)
Q1_agent.post=Q_after
Q1_agent.Q=Table() 
Q1_agent.learning=True

Q1_agent.α=0.3  
Q1_agent.ϵ=0.1  
Q1_agent.γ=0.9  

In [79]:
Q2_agent=Agent(Q_move)
Q2_agent.post=Q_after
Q2_agent.Q=Table()  
Q2_agent.learning=True

Q2_agent.α=0.3  
Q2_agent.ϵ=0.1  
Q2_agent.γ=0.9 

In [81]:
g=Game()
g.run(human_agent,human_agent)

====
Game  1
The players hands are [['8C', '3D'], ['9H']]
Player 1


hit or stay? hit


Player 1 moves hit


ValueError: not enough values to unpack (expected 3, got 2)